<a href="https://www.kaggle.com/code/majestyyolo/model-accuracy-comparison-on-social-network-ads?scriptVersionId=198697924" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Social Network Ads

## Table of contents
<ul>
    <li><a href = "#intro">Introduction</a></li>
    <li><a href = "#wrangling">Data Wrangling</a></li>
    <li><a href = "#preprocessing">Data Preprocessing</a></li>
    <li><a href = "#eda">Exploratory Data Analysis</a></li>
    <li><a href = "#features">Feature Engineering</a></li>
    <li><a href = "#modelling">Data Modelling</a></li>
    <li><a href = "#conclusion">Conclusions</a></li>
</ul>

___

<a id = "intro"></a>
## Introduction

### Dataset Description:
This dataset is a categorical dataset to determine whether a user purchased a particular product. Dataset was obtained from [kaggle](https://www.kaggle.com/datasets/rakeshrau/social-network-ads)
___

### Column Description:
- `User ID`: This is the User's unique Identification number
- `Gender`: Gender of the buyer
- `Age`: Age of the buyer
- `EstimatedSalary`: Estimated salary of the potential buyer
- `Purchased`: if the buyer purchased or not.

### Questions for Analysis:
- How many males purchased an item?
- How many females purchased an item?
- How many males didn't purchase an item?
- How many females didn't purchase an item?
- How many machine learning algorithms can accurately predict who will purchase an item or won't?

___

## Environment set-up

In [ ]:
# importing modules

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

<a id = "wrangling"></a>
___

## Data Wrangling

In this section, we will load our flat dataset from a csv file using `pandas` for further exploration

In [ ]:
import os
print(os.listdir('/kaggle/input/social-network-ads/'))

In [ ]:
# importing the dataset
data = pd.read_csv('/kaggle/input/social-network-ads/Social_Network_Ads.csv')

In [ ]:
# displaying the first five records of our dataset
data.head()

In [ ]:
# displaying all the records in our dataset
print(data.to_string())

In [ ]:
data.columns

In [ ]:
data.shape

In [ ]:
# value_counts() method is used to check the unique observations in a dataframe
data["Gender"].value_counts()

In [ ]:
labels = ["Female", "Male"]
colors = sns.color_palette('dark')[2:4]
data["Gender"].value_counts().plot(kind = "pie", colors = colors, autopct = "%1.1f%%", labels = labels)
plt.title("Gender Distribution")
plt.show()

In [ ]:
data["Purchased"].value_counts()

In [ ]:
colors = sns.color_palette("dark")[0:4]
data["Purchased"].value_counts().plot(kind = "pie", autopct = "%1.2f%%", colors = colors)
plt.title("Purchased Distribution")
plt.show()

In [ ]:
# showing the number of male employees that purchased an ad
data[(data["Gender"] == "Male") & (data["Purchased"] == 1)].shape[0]

___

`Q1` : How many males purchased an item?<br>
66 males purchased an item from an advertisement on social media

In [ ]:
# showing the number of male employees that did not purchased an ad
data[(data["Gender"] == "Male") & (data["Purchased"] == 0)].shape[0]

___

`Q3` : How many males didn't purchase an item?<br>
130 males did not purchase an item from advertisement on social media

In [ ]:
# showing the number of female employees that purchased an ad
data[(data["Gender"] == "Female") & (data["Purchased"] == 1)].shape[0]

___

`Q2` : How many females purchased an item?<br>
77 females purchased an item from advertisement on social media

In [ ]:
# showing the number of female employees that purchased an ad
data[(data["Gender"] == "Female") & (data["Purchased"] == 0)].shape[0]

___

`Q4` : How many females didn't purchase an item?<br>
127 females did not purchase an item from an advertisement on social media

In [ ]:
print(data.groupby(["Gender", "Purchased"]).size().unstack(fill_value = 0).to_string())

<a id = "preprocessing"></a>
___

## Data Preprocessing

In [ ]:
# Because machine learning modules understand numbers and not strings, we'll have to encode the Gender column

# creating a labelEncoder object
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

# invoking fit_tranform method on object
data["Gender"] = le.fit_transform(data["Gender"])

In [ ]:
data["Gender"].value_counts()

<a id = "eda"></a>
___
## Exploratory Data Analysis


In [ ]:
data.isnull().sum()

In [ ]:
data.info()

In [ ]:
data.describe()

<a id = "feature"></a>
___

## Feature Engineering
iGen: [1 - 24]<br>Millenials: [25 - 40]<br>GenX: [41 -55]<br>BabyBoomers: [56 - 75]<br>Silent Generation: [76 - 93]<br>Greatest Generation[94 and above]

In [ ]:
data['Age Category'] = [
     'Gen Z' if age <= 24  else
    'Millennial' if 25 <= age <= 40 else 
    'Gen X' if 41 <= age <= 55 else 
    'Baby Boomer' if 56 <= age <= 75 else
    'Silent Generation' if 76 <= age <= 93 else
    "Greatest Generation"
    for age in list(data['Age'].values)
]

data.head()

In [ ]:
# Using an income category [Inter-quatile range]
data["Income Category"] = pd.qcut(data["EstimatedSalary"], 3, labels = ["Low", "Medium", "High"])
data.head()

In [ ]:
age_cat = data.groupby("Age Category").count()
inc_cat = data.groupby("Income Category").count()

In [ ]:
plt.bar(age_cat.index.values, age_cat["Age"])
plt.ylabel("Number of Users")
plt.show()

In [ ]:
plt.bar(inc_cat.index.values, inc_cat["Age"])
plt.ylabel("Number of Users")
plt.show()

<a id = "model"></a>
___

## Model Building

In [ ]:
data.columns

In [ ]:
# importing libraries for machine learning
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix

# Splitting the data into train and test
x = data[["Gender", "Age", "EstimatedSalary"]]
y = data["Purchased"]

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.8, test_size = 0.2, random_state = 42)

### Naive Bayes

In [ ]:
# Fitting Naive Bayes to the training set

# importing Naive Bayes libraries
from sklearn.naive_bayes import GaussianNB

# Initialize the Naive Bayes model
NB_classifier = GaussianNB()

# Train the model
NB_classifier = NB_classifier.fit(x_train, y_train)

In [ ]:
# Make predictions
y_pred = NB_classifier.predict(x_test)

In [ ]:
# Evaluate the model performance
print(classification_report(y_test, y_pred))

In [ ]:
# printing the individual scores
NB_classifier_accuracy_score = accuracy_score(y_test, y_pred)
print(f"Accuracy Score:{NB_classifier_accuracy_score * 100:.0f}%\n")

NB_classifier_precision_score = precision_score(y_test, y_pred)
print(f"Precision Score:{NB_classifier_precision_score * 100:.0f}%\n")

NB_classification_recall_score = recall_score(y_test, y_pred)
print(f"Recall Score:{NB_classification_recall_score * 100:.0f}%\n")

NB_classifier_f1_score_score = f1_score(y_test, y_pred)
print(f"F1_Score:{NB_classifier_f1_score_score * 100:.0f}%")

In [ ]:
# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix using Seaborn's heatmap
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot = True, fmt = 'd', cmap = 'Blues', cbar = False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

In [ ]:
# importing libraries for roc and auc
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import RocCurveDisplay

# compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
print(f"ROC AUC: {roc_auc:.2f}")

In [ ]:
# plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, color = 'green', lw = 2, label = f"ROC curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')  # Diagonal line (naive bayes)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()

### Logistic Regression

In [ ]:
# Fitting Logistic Regression to the training set

# importing Logistic Regression libraries
from sklearn.linear_model import LogisticRegression

# Initialize the Logistic Regression model
LR_model = LogisticRegression(max_iter = 200, random_state = 42, C=0.5, solver='liblinear', penalty='l1')

# Train the model
LR_model = LR_model.fit(x_train, y_train)

In [ ]:
# Make predictions
y_pred = LR_model.predict(x_test)

In [ ]:
# Evaluate the model performance
print(classification_report(y_test, y_pred))

In [ ]:
# printing the individual scores
LR_model_accuracy_score = accuracy_score(y_test, y_pred)
print(f"Accuracy Score:{LR_model_accuracy_score * 100:.0f}%\n")

LR_model_precision_score = precision_score(y_test, y_pred)
print(f"Precision Score:{LR_model_precision_score * 100:.0f}%\n")

LR_model_recall_score = recall_score(y_test, y_pred)
print(f"Recall Score:{LR_model_recall_score * 100:.0f}%\n")

LR_model_f1_score_score = f1_score(y_test, y_pred)
print(f"F1_Score:{LR_model_f1_score_score * 100:.0f}%")

In [ ]:
# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix using Seaborn's heatmap
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='coolwarm', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

In [ ]:
# compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
print(f"ROC AUC: {roc_auc:.2f}")

In [ ]:
# plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, color = 'blue', lw = 2, label = f"ROC curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')  # Diagonal line (logistic regression)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()

### Random Forest

In [ ]:
# Fitting Random forest to training set

# importing Random forest libraries
from sklearn.ensemble import RandomForestClassifier

# Initialize the Random forest model
RF_classifier = RandomForestClassifier(n_estimators = 100, random_state = 42)

# Train the model
RF_classifier = RF_classifier.fit(x_train, y_train)

In [ ]:
# Make predictions
y_pred = RF_classifier.predict(x_test)

In [ ]:
# Evaluate the model performance
print(classification_report(y_test, y_pred))

In [ ]:
# printing the individual scores
RF_model_accuracy_score = accuracy_score(y_test, y_pred)
print(f"Accuracy Score:{RF_model_accuracy_score * 100:.0f}%\n")

RF_model_precision_score = precision_score(y_test, y_pred)
print(f"Precision Score:{RF_model_precision_score * 100:.0f}%\n")

RF_model_recall_score = recall_score(y_test, y_pred)
print(f"Recall Score:{RF_model_recall_score * 100:.0f}%\n")

RF_model_f1_score_score = f1_score(y_test, y_pred)
print(f"F1_Score:{RF_model_f1_score_score * 100:.0f}%")

In [ ]:
# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix using Seaborn's heatmap
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='magma', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

In [ ]:
# compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
print(f"ROC AUC: {roc_auc:.2f}")

In [ ]:
# plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, color = 'red', lw = 2, label = f"ROC curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')  # Diagonal line (random forest)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()

### XGBoost

In [ ]:
# Fitting XGBoost into the training set

# importing XGBoost libraries
import xgboost as xgb

# Initialize the XGBoost model
XGB_classifier = xgb.XGBClassifier(objective = 'multi:softmax', num_class = 3, random_state = 42)

# Train the model
XGB_classifier = XGB_classifier.fit(x_train, y_train)


In [ ]:
# Make predictions
y_pred = XGB_classifier.predict(x_test)

In [ ]:
# Evaluate the model performance
print(classification_report(y_test, y_pred))

In [ ]:
# printing the individual scores
XGB_classifier_accuracy_score = accuracy_score(y_test, y_pred)
print(f"Accuracy Score:{XGB_classifier_accuracy_score * 100:.0f}%\n")

XGB_classifier_precision_score = precision_score(y_test, y_pred)
print(f"Precision Score:{XGB_classifier_precision_score * 100:.0f}%\n")

XGB_classifier_recall_score = recall_score(y_test, y_pred)
print(f"Recall Score:{XGB_classifier_recall_score * 100:.0f}%\n")

XGB_classifier_f1_score_score = f1_score(y_test, y_pred)
print(f"F1_Score:{XGB_classifier_f1_score_score * 100:.0f}%")

In [ ]:
# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix using Seaborn's heatmap
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='viridis', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

In [ ]:
# compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
print(f"ROC AUC: {roc_auc:.2f}")

In [ ]:
# plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, color = 'yellow', lw = 2, label = f"ROC curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')  # Diagonal line (xgboost)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()

<a id = "conclusion"></a>

___
## Conclusion


- The dataset was evaluated using four machine learning models: `naive bayes`, `logistic regression`, `random forest` and `xgboost`.
- `naive bayes`: had an accuracy of `94%` and an AUC score of `0.93`.
- `logistic regression`: had an accuracy of `86%` and an AUC score of `0.82`.
- `random forest`: had an accuracy of `90%` and an AUC score of `0.90`.
- `xgboost`: had an accuracy of `88%` and an AUC score of `0.86`.

- Therefore `naive bayes` model performed the best with the best auc score.